In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

In [3]:
#for train
df['Age'] = 2019 - df['Year_of_birth']
df['Years_on_job'] = 2019 - df['Year_of_recruitment']
df['Age_of_recruitment'] = df['Year_of_recruitment']-df['Year_of_birth']
df['Training_score'] = df.Trainings_Attended*df.Training_score_average
# df['Performance_per'] = (df.Last_performance_score/12.5)*100
# df['Training_per'] = (df.Training_score_average/91)*100


#for test
test['Age'] = 2019 - test['Year_of_birth']
test['Years_on_job'] = 2019 - test['Year_of_recruitment']
test['Age_of_recruitment'] = test['Age']-test['Years_on_job']
test['Training_score'] = test.Trainings_Attended*test.Training_score_average
# test['Performance_per'] = (test.Last_performance_score/12.5)*100
# test['Training_per'] = (test.Training_score_average/91)*100

In [4]:
#train
df.Qualification.fillna('Non-University Education', inplace=True)

#test
test.Qualification.fillna('Non-University Education', inplace=True)

In [5]:
#train
df.drop(['Year_of_birth', 'Year_of_recruitment', 'State_Of_Origin'], axis=1, inplace=True)

In [6]:
#test
rem_test=['Year_of_birth', 'Year_of_recruitment', 'EmployeeNo', 'State_Of_Origin']

test.drop(rem_test, axis=1, inplace=True)

In [7]:
X = df.drop(['Promoted_or_Not', 'EmployeeNo'], axis=1)
y = pd.Series(df.Promoted_or_Not.values)

In [8]:
X.columns[X.dtypes == object]

Index(['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment',
       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers'],
      dtype='object')

In [9]:
X.columns

Index(['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment',
       'Trainings_Attended', 'Last_performance_score', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'Foreign_schooled',
       'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Age', 'Years_on_job', 'Age_of_recruitment', 'Training_score'],
      dtype='object')

##### for train

In [10]:
#label encoding categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
obj_cols = X.columns[X.dtypes == object]
for i in obj_cols:
    X[i] = le.fit_transform(X[i].astype(str))
ohe = OneHotEncoder(categorical_features = [0,1,2,3,9,10,11,12,13])
X = ohe.fit_transform(X).toarray()

y_le = LabelEncoder()
y = y_le.fit_transform(y)

c:\program files\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


##### for test

In [11]:
#label encoding categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
obj_cols = test.columns[test.dtypes == object]
for i in obj_cols:
    test[i] = le.fit_transform(test[i].astype(str))
ohe = OneHotEncoder(categorical_features = [0,1,2,3,9,10,11,12,13])
test = ohe.fit_transform(test).toarray()

c:\program files\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


_______________________

In [23]:
from imblearn.over_sampling import ADASYN
# Oversampling the dataset
sm = ADASYN(random_state=1)
X, y = sm.fit_sample(X, y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

#### XGBOOST MODEL

In [24]:
from xgboost import XGBClassifier

xgbm = XGBClassifier(learning_rate =0.021,
 n_estimators=100,
 max_depth=16,
 min_child_weight=0.535,
 gamma=0.05,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 scale_pos_weight=1,
 seed=27)


In [25]:
xgbm.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.05,
              learning_rate=0.021, max_delta_step=0, max_depth=16,
              min_child_weight=0.535, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, verbosity=1)

In [26]:
xgb_pred = xgbm.predict(X_test)

In [27]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
xgb_cm = confusion_matrix(y_test, xgb_pred)
print(xgb_cm)

[[6990   24]
 [ 462 6383]]


In [28]:
from sklearn.metrics import f1_score
f1_score(y_test, xgb_pred, average='micro')

0.9649325348149217

In [30]:
predicted = xgbm.predict(test)

In [31]:
ids = pd.read_csv('test.csv')['EmployeeNo'].values
output = pd.DataFrame({'EmployeeNo': ids, 'Promoted_or_Not': predicted})
output.to_csv("submission_new.csv", index=False)

In [21]:
df.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Last_performance_score,Targets_met,Previous_Award,Training_score_average,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not,Age,Years_on_job,Age_of_recruitment,Training_score
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,12.5,1,0,41,No,Married,No,No,0,0,33,8,25,82
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,12.5,0,0,52,Yes,Married,No,No,0,0,28,4,24,104
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,7.5,0,0,42,Yes,Married,No,No,0,0,32,7,25,84
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,2.5,0,0,42,Yes,Single,No,No,1,0,37,10,27,126
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,7.5,0,0,77,Yes,Married,No,No,1,0,29,7,22,231
